# subWESTnet 

A tool that will construct the downstream and/or upstream network from a specified utility.

----

### NOTE:

The network balancing uses the nodes calculated from only upstream and downstream and *does not* consider the entire network of utilities for the calculations. For example if a utility is chosen for a downstream graph, if it is not a source (i.e. in the middle of some path), it will not consider the utilities before it. As a result, the chosen utility's resulting energy is 0 kwh/af. This will also apply to children in the network that have one or more "parents" not directly linked to the original specified source. 

The same issue applies to upstream graphs, where some "parent" utility might serve other utilities downstream but is not linked to the utility that was initially chosen.

----

In [1]:
from sub_network import subWESTnet

Run the following cell to initialize the model. It takes two arguments, the first specifies the file path of the data; the second sorts the data by year for the calculations.

**Note:** I am creating two different instances because I dont want the graphs to be overwritten, since I'm using two examples.

In [2]:
upstream_example = subWESTnet('data/links_erl.csv', 2010)
downstream_example = subWESTnet('data/links_erl.csv', 2010)

The following table shows which methods to call on the model.

| Method           | Purpose                                                 | Returns |
|------------------|---------------------------------------------------------|---------|
|`.upstream(*)`   |Creates an upstram graph from `*`,<br> the name of a utility.|Nothing, but `Unweighted upstream graph built` will be <br> printed if the graph was successfully created.|
|`.downstream(*)` |Creates a downstream graph from `*`. | Similar statement printed (just corrected to downstream). |

Run the following cell so that the upstream network starting from the utility `1803033E` will be created.

In [3]:
upstream_example.upstream('1803033E')

Unweighted upstream graph built.


-----
In a similar fashion, running the following cell creates the downstream network from `SW1803SWP18`.

In [4]:
downstream_example.downstream('SW1803SWP18')

Unweighted downstream graph complete.


-----

After the network is created, you can either:

1. See the table of the utilities in the network 
2. Save the network to a dot file
3. Balance the graph

The following cell will show options 1 and 3.

In [5]:
# Balance graph -- when complete, it will print "Graph is weighted"
upstream_example.balance_graph()

# Table of utilities
upstream_example.table()

Graph is weighted.


,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
0,SW_CVPFKC,1803033E,53121.00000,23.220000,0.0,53121.00
1,1803033GW,1803033PD,30799.03923,399.000000,26.0,29572.94
2,FRES_ID,1803033PD,19023.93641,87.000000,224.0,18266.60
3,SW_CVPFKC,1803033PD,14701.51218,23.220000,224.0,14116.25
4,RES_MLRTN,SW_CVPFKC,67956.99412,0.000000,0.0,67371.23
5,1803033PD,1803033E,61955.79000,17.600000,0.0,61955.79
6,1803033REC,1803033NPD,88.00000,0.000000,236.0,88.00
7,1803033NPD,1803033E,88.00000,21.120000,0.0,88.00
8,GW1803-5-22.08,1803033GW,30799.03923,0.000000,0.0,29572.94
9,GW1803-5-22,GW1803-5-22.08,51162.00619,0.000000,0.0,49779.01


When balancing the graph, a table with the volume weighted averages is created and can be saved, like the original WESTnet tool. In order to save the table, call `upstream_example.save_energy_df(filename)`, where filename is a string of the csv name.

The cell below saves the table (commented out), and will also show the table for the upstream example.

In [17]:
# I commented this out because I didn't want extra files to be in the same directory :)
#upstream_example.save_energy_df('1803033E_volavgs')

upstream_example.energy

,node,kwh/af
0,WWT1803033,0.000000
1,FRES_ID,0.000000
2,RES_MLRTN,0.000000
3,1803033REC,0.642618
4,1803033NPD,236.642618
5,GW1803-5-22,0.000000
6,GW1803-5-22.08,0.000000
7,GWR1803-5-22.08,1844.642618
8,SW_CVPFKC,0.000000
9,1803033GW,604.416306


After the graph is balanced, the graph can be saved to a dot file! In order to save the dot file, call `.to_dot()`. It will save to the output folder in your current directory by default.

**Note:** If the graph isn't balanced, the resulting network will have uncolored nodes.

In [18]:
# will have colors
upstream_example.to_dot()

# won't have colors
downstream_example.to_dot()

In order to see the graph, copy the code in the saved dot file and paste it in the box on this [website](https://dreampuf.github.io/GraphvizOnline/). You can save the file in various formats.

That's all the code needed to use the subWESTnet tool!

----

Below, there is an explanation on how to create a similar network. Instead of using the code in the previous cells, this method uses the original tool (which uses all utilities in the balancing calculations).

In [10]:
from build import WESTNet
import network as net

# Initialize model
w = WESTNet('data/links_erl.csv')

# balance graph
w.run(2010, pdf = True, dot=False)

In order to create the upstream or downstream network, use `.create_subgraph(...)`. The arguments are as follows

1. **Utility (required)** - A string of the utility name
2. **Filename (required)** - The name that the dot file will be saved to. Will be saved in the `output` folder.
3. **Up (default=False)** -  This is if you want an upstream or downstream graph.
4. **csv (default=True)** -  If true, the table of volume weighted averages will be saved to a csv.

The following cell will save the upstream graph from `1803033E` and save the resulting network to the specified file name. The csv file of weighted averages will not be saved, but will be shown in the output.

In [11]:
w.create_subgraph('1803033E', '1803033E_upstream_full_balance', up=True, csv=False)

,node,kwh/af
0,GWR1803-5-22.08,1844.642618
1,WWT1803033,0.000000
2,1803033NPD,236.642618
3,GW1803-5-22.08,0.000000
4,1803033E,392.483777
5,1803033GW,604.416306
6,1803033REC,0.642618
7,1803033PD,639.384932
8,SW_CVPFKC,0.000000
9,GW1803-5-22,0.000000


Like before, if you want to see the network, go to the dot file and paste the code in the dot file [here](https://dreampuf.github.io/GraphvizOnline/).

----